In [31]:
import json
import os
import random
import shutil
import sys
import time
from collections import defaultdict
from itertools import chain, cycle, islice, product
from typing import Annotated, Any

import einops as ein
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from beartype import beartype as typed
from beartype.door import die_if_unbearable as assert_type
from beartype.typing import Callable, Iterable
from beartype.vale import Is
from jaxtyping import Bool, Float, Int
from numpy import ndarray as ND
from tqdm import tqdm

Country = str
Metric = str
Year = int

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
data = pd.read_csv("data.csv", index_col=0)

In [33]:
@typed
def get_metric(
    metric: Metric,
    country: Country,
    start_year: Year | None = None,
    end_year: Year | None = None,
) -> ND:
    mask = data["country"] == country
    if start_year is not None:
        mask = mask & (data["year"] >= start_year)
    if end_year is not None:
        mask = mask & (data["year"] <= end_year)
    years = data[mask]["year"].to_list()
    if start_year is not None:
        assert min(years) == start_year
    if end_year is not None:
        assert max(years) == end_year
    return np.array(data[mask][metric])

In [58]:
@typed
def fuzzy_country(country: str) -> str:
    candidates = set()
    for c in data["country"]:
        if country in c:
            candidates.add(c)
    print(candidates)
    assert len(candidates) == 1
    return next(iter(candidates))

@typed
def country_mask(country:str) -> pd.Series:
    full_name = fuzzy_country(country)
    return data["country"] == full_name

In [60]:
data[(data["year"] == 2021) & country_mask("Korea (Republic")][["country", "hdi"]].sort_values("hdi", ascending=False).head(60)

{'Korea (Republic of)'}


,country,hdi
3039,Korea (Republic of),0.925


In [ ]:
get_metric("hdi", "Ru", 2017, 2020)